In [1]:
%load_ext autoreload
%autoreload 2

from kasearch import AlignSequences, SearchDB, ExtractMetadata, PrepareDB

## Prepare query sequence (sequence to search with)

In [2]:
raw_queries = [
    'QVKLQESGAELARPGASVKLSCKASGYTFTNYWMQWVKQRPGQGLDWIGAIYPGDGNTRYTHKFKGKATLTADKSSSTAYMQLSSLASEDSGVYYCARGEGNYAWFAYWGQGTTVTVSS',
    'EVQLQQSGTVLARPGASVKMSCEASGYTFTNYWMHWVKQRPGQGLEWIGAIYPGNSDTSYIQKFKGKAKLTAVTSTTSVYMELSSLTNEDSAVYYCTLYDGYYVFAYWGQGTLVTVSA',
]

In [3]:
query_db = AlignSequences(raw_queries, # Sequences as strings to align.
                          n_jobs=1     # Allocated number for jobs/threads for the search.
                         )
query_db.db.aligned_seqs

array([[81, 86, 75,  0, 76, 81, 69, 83, 71, 65,  0, 69, 76, 65, 82, 80,
        71, 65, 83, 86, 75, 76, 83, 67, 75, 65, 83, 71, 89, 84, 70,  0,
         0,  0,  0,  0,  0,  0,  0,  0, 84, 78, 89, 87, 77, 81,  0, 87,
        86, 75, 81,  0, 82,  0, 80,  0, 71,  0, 81,  0,  0, 71,  0, 76,
        68,  0, 87, 73, 71, 65, 73, 89, 80, 71,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0, 68, 71, 78, 84, 82, 89,  0,  0, 84,  0,  0,
        72,  0,  0, 75, 70,  0,  0, 75,  0,  0,  0, 71, 75, 65, 84, 76,
        84, 65,  0, 68,  0,  0,  0, 75,  0, 83,  0,  0, 83, 83,  0,  0,
         0,  0, 84,  0, 65, 89, 77, 81, 76, 83, 83, 76, 65, 83,  0, 69,
        68, 83, 71, 86, 89, 89, 67, 65, 82, 71, 69, 71, 78,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0, 89, 65, 87, 70, 65, 89, 87, 71,  0, 81,
        71, 84, 84, 86, 84, 86, 83, 83],
       [69, 86, 81,  0, 76, 81, 81, 83, 71, 84,  0, 86, 76, 65, 82, 80,
        71, 65, 83, 86,

--------------
## Initiate search class 

- Default will download a prepared version of OAS to search against
- Alternatively, you can give it the path for a custom database to search against. (See below for how to create a custom database).

In [4]:
#DB_PATH = "/data/icarus/olsen/projects/ka-search-project/reports/oasdb/oasdb_20220611/"
DB_PATH = "../data/db-example"

oasdb = SearchDB(database_path=DB_PATH,       # Path to your database. Default will be to download a prepared version of OAS.
                 allowed_chain='Any',       # Search against a specific chain. Default is any chain.
                 allowed_species='Any',       # Search against a specific species. Default is any species.
                 n_jobs=5                     # Allocated number for jobs/threads for the search.
                )

-----------
## Run search

A search against all of OAS takes ~2min for each sequence.


In [5]:
%%time 
oasdb.search(query_db.db.aligned_seqs[0], # Input can only be a single aligned sequence at a time.
             keep_best_n=2,               # You can define how many most similar sequences to return
             reset_best=True              # In cases where you want to search with the same sequence against multiple databases, you might want to not reset_best. (True is default)
            )

CPU times: user 43.8 ms, sys: 2.07 ms, total: 45.9 ms
Wall time: 1.01 s


### Get N best identities

Identities of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command.

In [6]:
oasdb.current_best_identities

array([[0.53370893, 0.4642857 , 0.5833333 ],
       [0.53370893, 0.35714287, 0.5833333 ]], dtype=float32)

### Get ID's of sequences with the N best identities

ID's of the most similar sequence (1st column), CDRs (2nd column) and CDR3 (3rd column) in all of OAS, can be fetched from the object with the bellow command. 

With ExtractMetadata, these ID's can be used to extract the sequence and its meta data (species, isotype, whole sequence etc.).

In [7]:
oasdb.current_best_ids

array([[[   6,  365],
        [   9, 1330],
        [   1,    0]],

       [[   7, 1705],
        [   9, 1343],
        [   1,  229]]], dtype=int32)

---------
## Extract the meta data from matched sequences


The meta data for the N best of the different regions, can be extracted with the following indexes.
- 0: whole sequence
- 1: CDRs
- 2: CDR3

e.g. "oasdb.current_best_ids\[:,0\]\" returns meta data for the N best whole sequence identities.

NB: The column "sequence_alignment_aa" holds the antibody sequence.

In [8]:
meta_db = ExtractMetadata()

In [9]:
n_best_sequences = meta_db.get_meta(oasdb.current_best_ids[:,0])
n_best_sequences

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,BType,Vaccine,Disease,Subject,Longitudinal,Organism,Unique sequences,Total sequences,Isotype,Chain
0,TGGGGTTTTGATCCTGAAGATGGTGAAACAATATACGCACAGAAGT...,H,F,T,F,T,T,F,IGHV1-24*01,IGHD3-10*01,...,ASC,None,SARS-COV-2,Patient-1,no,human,1184,1602,IGHG,Heavy
1,GTGGCTGGGAAGGACATACTACAGGTCCAAGTGGTATAATGAATAT...,H,F,T,F,T,F,F,IGHV6-1*01,NaN,...,ASC,None,SARS-COV-2,Patient-1,no,human,3323,6909,IGHM,Heavy


In [10]:
n_best_sequences.sequence_alignment_aa.values

array(['GFDPEDGETIYAQKFQGRVTMTEDTSTDTAYMELSSLRSEDTAVYYCATASPITIHHNWFDPWGQGTLVTVSS',
       'WLGRTYYRSKWYNEYAVSVKGRITINPDTSKNQFSLQLNSVTPEDTAVYYCSRARDGYVDFWGQGTLVTVSS'],
      dtype=object)

----------
## Create custom database


To create your own database you first need to create a csv file in the OAS format. For an example file, look at data/custom-data-example.csv. This file consists of a dictionary containing the metadata in the first line and then rows of the individual sequences afterwards. Only the Species and Chain is strictly needed in the metadata, and only the amino acids sequence of the antibodies is required for each antibody sequence.

### 1. Format your data into OAS files

In [11]:
import json
import os
import pandas as pd

from kasearch.species_anarci import number
from kasearch.merge_db import merge_files

In [12]:
metadata = {"Species":"Human", "Chain":"Heavy"}
metadata = pd.Series(name=json.dumps(metadata), dtype='object')
seqsdata = pd.DataFrame([["EVQLVESGGGLAKPGGSLRLHCAASGFAFSSYWMNWVRQAPGKRLEWVSAINLGGGLTYYAASVKGRFTISRDNSKNTLSLQMNSLRAEDTAVYYCATDYCSSTYCSPVGDYWGQGVLVTVSS"],
                          ["EVQLVQSGAEVKRPGESLKISCKTSGYSFTSYWISWVRQMPGKGLEWMGAIDPSDSDTRYNPSFQGQVTISADKSISTAYLQWSRLKASDTATYYCAIKKYCTGSGCRRWYFDLWGPGT"]
                         ], columns = ['sequence_alignment_aa'])

In [13]:
save_file = "../data/custom-data-example-2.csv"
metadata.to_csv(save_file, index=False)
seqsdata.to_csv(save_file, index=False, mode='a')

### 2. Turn your OAS formatted files into a custom database

After creating all the files you want to include in the new database, you can run the following code to create the database.

In [14]:
db_folder = "../data/my_db"
db_files = ['../data/custom-data-example.csv']

In [15]:
newDB = PrepareDB(db_path=db_folder)

db_dict = {}
for num, data_unit_file in enumerate(db_files):

    db_dict[num] = data_unit_file
    
    metadata = json.loads(','.join(pd.read_csv(data_unit_file, nrows=0).columns))
    seqsdata = pd.read_csv(data_unit_file, header=1, usecols=['sequence_alignment_aa']).iloc[:,0].values
    numbered_sequences = [number(sequence, allowed_species=None)[0] for sequence in seqsdata]

    newDB.prepare_database(numbered_sequences, 
                           file_id=num, 
                           chain=metadata['Chain'], 
                           species=metadata['Species'])
    
newDB.save_database()

merge_files(db_folder)

with open(os.path.join(db_folder, "my_db_id_to_study.txt"), "w") as handle: handle.write(str(db_dict))

### 3. Initiate the search class with your custom database


In [16]:
mydb = SearchDB(database_path=db_folder,      # Path to your database. Default will be to download a prepared version of OAS.
                 allowed_chain='Heavy',       # Search against a specific chain. Default is any chain.
                 allowed_species='Any',       # Search against a specific species. Default is any species.
                 n_jobs=1                     # Allocated number for jobs/threads for the search.
                )